In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [2]:
# Define variable here

# Observed big difference in obtained minimum when increasing the number of reads from 100 to 1000
num_reads = 1000
num_events = 1000
path = '/home/andrea/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [88]:
for ev in range(0, 10): #num_events):
    print(ev)

    fname = path + 'Event_' + str(ev) + '.dat'
    f = open(fname, 'r')

    pat = re.compile(r"\s+")

    g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

    px = np.array([ float(x) for x in g[0]])
    py = np.array([ float(x) for x in g[1]])
    pz = np.array([ float(x) for x in g[2]])
    e = np.array([ float(x) for x in g[3]])

    # Generate qubo

    n_part = len(px)
    #print(n_part)
    s = Array.create('s', shape=n_part, vartype='BINARY')

    # Create an array of zeroes for qubo matrix coefficients
    coeff = [[0] * n_part for _ in range(n_part)] 

    for i in range(0,n_part):
        for j in range(0,n_part):
            coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

    # Construct Hamiltonian
    H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

    # Compile model using pyqubo

    model = H.compile()
    qubo, offset = model.to_qubo()
    bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)

    sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])

    elite_threshhold = (-1.0*sum_mom*sum_mom)/6.

    new_energy = 0.0

    for j in range(0,5):
        print("Iteration number %s" %j)
        # Submit problem to D-Wave and sample for a relatively small amount of times
        response = sampler.sample(bqm, num_reads=100)
        if (response.first.energy > new_energy): break
        print("Energy: %s" %response.first.energy)
        
        for p in range(n_part):
            key = 's[%s]' %p
            val_list = []
            for datum in response.data(['sample','energy']):
            #For now, lets fix the elite_threshold percentile to have energies at least half of the sum of the momentum in the event.
                if(datum.energy < elite_threshhold):
                    if(datum.sample[key]): val_list.append(datum.sample[key])
            x = sum(val_list)
            if(x>650):
                bqm.fix_variable(key, 1)
        new_energy = response.first.energy 


0
Iteration number 0
Energy: -2012.0070412910209
Iteration number 1
Energy: -2012.0070412910209
Iteration number 2
Energy: -2012.0070412910209
Iteration number 3
1
Iteration number 0
Energy: -1951.4651314954767
Iteration number 1
Energy: -1955.3636079430344
Iteration number 2
2
Iteration number 0
Energy: -734.3065821813152
Iteration number 1
Energy: -796.7852276565509
Iteration number 2
Energy: -798.1794953477629
Iteration number 3
Energy: -804.854241068897
Iteration number 4
3
Iteration number 0
Energy: -1646.2135229366816
Iteration number 1
Energy: -1675.6285802622076
Iteration number 2
Energy: -1697.2616913832464
Iteration number 3
4
Iteration number 0
Energy: -1809.3019318874638
Iteration number 1
Energy: -1848.3703614818182
Iteration number 2
5
Iteration number 0
Energy: -1810.263170471561
Iteration number 1
Energy: -1819.8674550896062
Iteration number 2
6
Iteration number 0
Energy: -1913.4723156340842
Iteration number 1
Energy: -1920.9398903460826
Iteration number 2
7
Iteration n

In [90]:
thrust = 2.*np.sqrt(804.854241068897)/sum_mom
print(thrust)

0.625597623042203


In [45]:
#SPVAR Algorithm

# - Obtain a sample of sample_size from the sampler
# - Narrow down the solutions to the elite_threshhold percentile
# - Find the mean value of each variable in all solutions
# - Fix the variables for which the mean absolute value is larger than fixing_threshold
# - Update J and h

mean_val = {}
print(-1.0*sum_mom*sum_mom/8.)
elite_threshhold = (-1.0*sum_mom*sum_mom)/6.

for p in range(n_part):
    key = 's[%s]' %p
    val_list = []
    for datum in response.data(['sample','energy']):
        #print(datum.energy)
        #For now, lets fix the elite_threshold percentile to have energies at least half of the sum of the momentum in the event.
        if(datum.energy < elite_threshhold):
            val_list.append(datum.sample[key])
            #f.write("%s\n" %(response.first.sample[key]))
    mean_val[key] = val_list 


-1032.081450522637


In [67]:
import statistics

for p in range(n_part):
    key = 's[%s]' %p
    x = sum(mean_val[key])
    if(x>650): print(x, key)

703 s[5]
668 s[11]


In [65]:
n_part

28

In [68]:
bqm.fix_variable('s[5]', 1)

In [69]:
bqm.fix_variable('s[11]', 1)

In [71]:
response.first.energy

-2005.200790919585

In [75]:
response = sampler.sample(bqm, num_reads=100)


In [76]:
response.first.energy

-2012.0050389347948